Імпортувати основні бібліотеки

In [72]:
import pandas as pd
from pathlib import Path
import re


In [73]:
path = Path("data/Кладовище.xlsx")

dfs = []
with pd.ExcelFile(path, engine="openpyxl") as xls:
    for sh in xls.sheet_names:
        df = pd.read_excel(
            xls,
            sheet_name=sh,
            header=None,
            dtype="string"
        )
        if df.empty:
            continue

        # Нормалізуємо до рівно 5 колонок і задаємо імена
        df = df.reindex(columns=range(4))
        df.columns = ["number", "name", "dates", "teritory"]

        # Тягнемо вниз лише службові колонки — БЕЗ 'name'
        # fill_cols = ["number", "dates", "teritory"]
        # df[fill_cols] = df[fill_cols].ffill()

        # Назва аркуша
        df["sheet"] = sh

        # Просто перевірка, що аркуш не повністю порожній (не ховає порожні name)
        if not df.dropna(how="all").empty:
            dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

# Як і раніше: number не потрібен у вивантаженні
df_all.drop(columns=["number"], inplace=True)


C:\Users\ПК\AppData\Local\Temp\ipykernel_8808\3462611488.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)


### Заповнити пробіли псевдоданими

In [74]:
# Заповнити пропуски псевдоданими
# df_all['full_name'] = df_all["full_name"].fillna("John_Dou") #Заповнити пропуски в іменах
df_all['teritory'] = df_all['teritory'].fillna("To Be Confirmed") 
df_all.isna().sum()



name         113
dates       1545
teritory       0
sheet          0
dtype: int64

In [75]:
# Додати колонку з кількістю слів в датафрейм, в майбутньому для розділення імені

word_pattern = r"[A-Za-zА-Яа-яІіЇїЄєҐґ’']+" #будь яка послідовність кирилиці або латиниці є словом

# колонка з кількістю слів
df_all["word_count"] = (
    df_all["name"]  
      .fillna(" ") #Заповнити null щоб не падало (просто страховка, вище замінив на J Dou])
      .str.replace(r"\s+", " ", regex=True).str.strip() # прибрати зайві пробіли
      .str.findall(word_pattern) # знайти матчі
      .str.len() # порахувати матчі
)


In [76]:
# Проставити імя та призвіще
norm = (
    df_all["name"]
      .fillna("")
      .str.replace(r"\s+", " ", regex=True).str.strip()
)


df_all["second_name"]  = norm.str.extract(rf"^({word_pattern})", expand=False)
df_all["first_name"] = norm.str.extract(rf"^(?:{word_pattern})\s+({word_pattern})", expand=False)

    


In [77]:
# Створення колонок з датою народження та датою смерті
s = df_all["dates"].astype("string").str.replace(r"[–—−]", "-", regex=True)

# витягуємо ПЕРШИЙ рік до тире (народження) і ПЕРШИЙ рік після тире (смерті)
rng = s.str.extract(r'^\s*(?:.*?(\d{3,4})).*?-\s*(?:.*?(\d{3,4})).*$')

df_all["birth_year"] = rng[0]
df_all["death_year"] = rng[1]

# якщо в комірці був лише один рік без тире — це рік смерті
single = s.str.extract(r'^\s*(?:.*?(\d{3,4}))\s*$')
df_all["death_year"] = df_all["death_year"].fillna(single[0])

# привести до числового типу з підтримкою NaN
for col in ["birth_year", "death_year"]:
    df_all[col] = pd.to_numeric(df_all[col], errors="coerce").astype("Int64")

df_all.to_excel("test.xlsx", index=False)
